Download Price Paid Data and Load into DB

In [1]:
# Verifies that CSV file is up-to-date (else: D/L from Land Registry server)

from ppc_db import fetch_csv

url = 'http://prod.publicdata.landregistry.gov.uk.s3-website-eu-west-1.amazonaws.com/pp-complete.csv'
dest = 'C:/Users/jamie/OneDrive/Python/Py_24/Docker/Data/'

file_name = fetch_csv(url,dest)

100%|██████████| 5.17G/5.17G [05:59<00:00, 14.4MB/s]


In [2]:
# Copy local file into Docker volume

from ppc_db import copy_file_to_docker_volume

mount_path = "/var/lib/mysql-files"

copy_file_to_docker_volume(file_name+".csv",
                           container_name = "110e4f0dab4e",
                           local_path = "C:/Users/jamie/OneDrive/Python/Py_24/Docker/Data/",
                           mount_path = mount_path
                           )

Successfully copied C:/Users/jamie/OneDrive/Python/Py_24/Docker/Data/ to path /var/lib/mysql-files (container 110e4f0dab4e)


In [3]:
# CREATE TABLE (SQL)

from ppc_db import sql_commitchanges

#file_name = 'Sep24pricepaidcomplete'

db_args = {
    'user':'root',
    'password':'password',
    'host':'localhost',
    'port':3306,
    'database':'CSV_DB 7'
 }

base = 'CREATE TABLE'+' `'+file_name+'` '

fd = open('tbl_params.sql','r')
sqlFile = fd.read()
fd.close()

sql_commitchanges(query=base+sqlFile,**db_args)

In [4]:
# LOAD DATA (SQL)

#mount_path = "/var/lib/mysql-files"

start = "LOAD DATA INFILE "+"\""+mount_path+"/"+file_name+".csv"+"\" "
mid = "INTO TABLE "+file_name+" "
end = "COLUMNS TERMINATED BY ',' OPTIONALLY ENCLOSED BY '\"' ESCAPED BY '\"' LINES TERMINATED BY '\\n'"

sql_commitchanges(start+mid+end, **db_args)